<a href="https://colab.research.google.com/github/huongtn1112/BTL_ML_Re-Id/blob/main/Re_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:
! git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 9809 (delta 213), reused 301 (delta 206), pack-reused 9487
Receiving objects: 100% (9809/9809), 9.57 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (7221/7221), done.


In [5]:
cd deep-person-reid/

/content/deep-person-reid


In [6]:
! pip install -r requirements.txt

     |████████████████████████████████| 5.8 MB 9.4 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 185 kB 54.1 MB/s 
     |████████████████████████████████| 42 kB 932 kB/s 
     |████████████████████████████████| 42 kB 805 kB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.1
    Uninstalling importlib-metadata-4.8.1:
      Successfully uninstalled importlib-metadata-4.8.1


In [7]:
!pip install torch torchvision torchaudio

     |████████████████████████████████| 2.9 MB 13.0 MB/s 
     |████████████████████████████████| 1.9 MB 37.8 MB/s 
     |████████████████████████████████| 1.9 MB 30.5 MB/s 


In [8]:
!python setup.py develop

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
writing manifest file 'torchreid.egg-info/SOURCES.txt'

# Import torchreid

In [9]:
import torchreid

# Load data manager

In [10]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3483 KB/s, 42 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




# Build model, optimizer and lr_scheduler

In [11]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# Build engine

In [12]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

# Run training and test

In [13]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=40,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: [1/40][10/404]	time 0.432 (1.016)	data 0.002 (0.558)	eta 4:33:36	loss 7.0034 (6.7187)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/40][20/404]	time 0.433 (0.726)	data 0.000 (0.280)	eta 3:15:22	loss 6.6577 (6.9014)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/40][30/404]	time 0.440 (0.630)	data 0.002 (0.187)	eta 2:49:20	loss 6.8844 (6.8717)	acc 3.1250 (1.2500)	lr 0.000300
epoch: [1/40][40/404]	time 0.438 (0.582)	data 0.000 (0.140)	eta 2:36:19	loss 6.9013 (6.8126)	acc 0.0000 (1.1719)	lr 0.000300
epoch: [1/40][50/404]	time 0.441 (0.553)	data 0.002 (0.112)	eta 2:28:29	loss 6.6376 (6.7813)	acc 3.1250 (1.1250)	lr 0.000300
epoch: [1/40][60/404]	time 0.438 (0.534)	data 0.001 (0.094)	eta 2:23:16	loss 6.5208 (6.7459)	acc 3.1250 (1.0417)	lr 0.000300
epoch: [1/40][70/404]	time 0.440 (0.520)	data 0.000 (0.081)	eta 2:19:29	loss 6.8344 (6.7268)	acc 0.0000 (0.9821)	lr 0.000300
epoch: [1/40][80/404]	time 0.438 (0.510)	data 0.001 (0.071)	eta 2:16:44	loss 6.4494 (6.7049)	acc 0.0000 (0.8984)	lr 0.000300
